## Finite state machines

In this notebook, we attempt to translate finite state machines into reservoir memory machines. We have two variations of this task. First, directly translate a known finite state machine into a reservoir memory machine and predict the correct output on long test sequences. Second, we generate all sequences of length up to the number of states+1, then infer a finite state machine from that training data using classical methods, and translate this FSM into a reservoir memory machine, which introduces another layer of complication.

In [1]:
import random
import numpy as np

task = 'fsms'

# set up experimental hyperparameters
num_states = 4
num_in_symbols = 2
num_out_symbols = 2
# the number of test time series in each repeat
N_test = 10

# as training data for FSM inference, sample all possible input sequences of length
# num_states + 1 over num_in_symbols
Xs_inference = []
stk = [[]]
while stk:
    prefix = stk.pop()
    if len(prefix) == num_states + 1:
        Xs_inference.append(prefix)
        continue
    for x in range(num_in_symbols):
        stk.append(prefix + [x])

import os
# for evaluation, sample some long input sequences over the input alphabet and store them
# for fair comparisons later on
eval_seq_len = 256
eval_seq_path = '%s_evaluation_sequences.csv' % task
if os.path.isfile(eval_seq_path):
    Xs_test = np.loadtxt(eval_seq_path, delimiter='\t', dtype=int).T
else:
    Xs_test = np.random.randint(num_in_symbols, size=(N_test, eval_seq_len))
    np.savetxt(eval_seq_path, Xs_test.T, delimiter='\t', fmt='%d')

# transform to one-hot-coding
Xs_test_one_hot = []
for X in Xs_test:
    X_oh = np.zeros((len(X), num_in_symbols))
    for t in range(len(X)):
        X_oh[t, X[t]] = 1.
    Xs_test_one_hot.append(X_oh)

import fsm
# set up a function to generate training data from a finite state machine,
# either for the hard or the easy task
def generate_seq(delta, rho, with_inference = False):
    # if we wish to use automaton inference, first learn a surrogate automaton instead
    # of the actual automaton
    if with_inference:
        # generate training data for automaton learning
        Ys, _ = fsm.label_sequences(Xs_inference, delta, rho)
        # learn the surrogate automaton
        delta, rho = fsm.learn_fsm(Xs_inference, Ys)
    # then generate training data, in particular all paths with at most one cycle
    # through the finite state machine
    paths = fsm.one_cyclic_paths(delta)
    # convert them to training data for a reservoir memory machine
    Xs = []
    Qs = []
    Ys = []
    for path in paths:
        # initialize input matrix, state array, and output array
        X = np.zeros((len(path), delta.shape[1]))
        Q = np.zeros(len(path))
        Y = np.zeros((len(path), 1))
        # convert the path
        for t in range(len(path)):
            q, x = path[t]
            X[t, x] = 1.
            Q[t] = q + 1
            Y[t] = rho[q]
        # append to training data
        Xs.append(X)
        Qs.append(Q)
        Ys.append(Y)
    if with_inference:
        return Xs, Qs, Ys, delta, rho
    else:
        return Xs, Qs, Ys

## Hyperparameter Optimization

In [2]:
# set the hyper-parameter ranges for all models
models = ['ESN', 'CRJ', 'LMU', 'RMM_ESN', 'RMM_CRJ', 'RMM_LMU']
# all models get the same number of neurons
m = 128
hyperparam_ranges = {
    'ESN' : {
        'radius' : [0.5, 0.7, 0.9],
        'sparsity' : [0.1, 0.2, 0.5],
        'regul' : [1E-7, 1E-5, 1E-3]
    },
    'CRJ' : {
        'v' : [0.1, 0.3, 0.5],
        'w_c' : [0.1, 0.7, 0.9],
        'w_j' : [0.1, 0.2, 0.4],
        'l' : [4, 8, 16],
        'regul' : [1E-7, 1E-5, 1E-3]
    },
    'LMU' : {
        'regul' : [1E-7, 1E-5, 1E-3]
    },
    'RMM_ESN' : {
        'radius' : [0.5, 0.7, 0.9],
        'sparsity' : [0.1, 0.2, 0.5],
        'regul' : [1E-7, 1E-5, 1E-3],
        'C' : [1., 100., 10000.]
    },
    'RMM_CRJ' : {
        'v' : [0.1, 0.3, 0.5],
        'w_c' : [0.1, 0.7, 0.9],
        'w_j' : [0.1, 0.2, 0.4],
        'l' : [4, 8, 16],
        'regul' : [1E-7, 1E-5, 1E-3],
        'C' : [1., 100., 10000.]
    },
    'RMM_LMU' : {
        'regul' : [1E-7, 1E-5, 1E-3],
        'C' : [1., 100., 10000.]
    }
}

import esn
import crj
import lmu
import rmm

# set up a function to initialize an instance for each model
def setup_model(model, hyperparams):
    # first, set up the correct reservoir and nonlinearity
    if model.endswith('ESN'):
        U, W = esn.initialize_reservoir(m, num_in_symbols, radius = hyperparams['radius'], sparsity = hyperparams['sparsity'])
        nonlin = np.tanh
    elif model.endswith('CRJ'):
        U = crj.setup_input_weight_matrix(num_in_symbols, m, v = hyperparams['v'])
        W = crj.setup_reservoir_matrix(m, w_c = hyperparams['w_c'], w_j = hyperparams['w_j'], l = hyperparams['l'])
        nonlin = np.tanh
    elif model.endswith('LMU'):
        degree = int(m/num_in_symbols)-1
        U, W = lmu.initialize_reservoir(num_in_symbols, degree, num_states+1)
        nonlin = lambda x : x
    else:
        raise ValueError('Unknown model: %s' % model)
    # then, set up the model
    if not model.startswith('RMM_'):
        net = esn.ESN(U, W, regul = hyperparams['regul'], input_normalization = False, nonlin = nonlin)
    else:
        net = rmm.RMM(U, W, regul = hyperparams['regul'], input_normalization = False, nonlin = nonlin, C = hyperparams['C'], q_0 = 1, discrete_prediction = True, svm_kernel = 'rbf')
    return net

In [3]:
# perform a hyperoptimization where we test R random hyperparameter
# settings for each model and perform num_repeats repeats to obtain
# statistics. The hyperparameters with the best mean performance across
# repeats will be selected

hyper_R = 10
hyper_num_repeats = 10

# try to load the selected hyperparameters from file
import json
import os
import random

hyperparam_path = '%s_hyperparams.json' % task
if os.path.isfile(hyperparam_path):
    with open(hyperparam_path, 'r') as hyperparam_file:
        hyperparams = json.load(hyperparam_file)
else:
    # generate random parameter combination for all models
    hyperparams = {}
    for model in models:
        hyperparams[model] = []
        for r in range(hyper_R):
            params_r = {}
            hyperparams[model].append(params_r)
            # sample a novel random combination of hyper parameters
            # for the current model
            for key in hyperparam_ranges[model]:
                param_range = hyperparam_ranges[model][key]
                value = param_range[random.randrange(len(param_range))]
                params_r[key] = value
            # set up an extra key for the errors
            params_r['errors'] = []

    for repeat in range(hyper_num_repeats):
        print('--- repeat %d of %d ---' % (repeat+1, hyper_num_repeats))
        # sample a finite state machine but exclude trivial ones
        while True:
            delta, rho = fsm.sample_fsm(num_states, num_in_symbols, num_out_symbols)
            if len(np.unique(rho)) > 1:
                break
        # generate the according training data
        Xs, Qs, Ys = generate_seq(delta, rho)
        # now iterate over all models
        for model in models:
            print('-- model: %s --' % model)
            # and iterate over all parameter combinations for this model
            min_rmse = np.inf
            for params_r in hyperparams[model]:
                # set up a model instance
                net = setup_model(model, params_r)
                # fit the model to the data
                if model.startswith('RMM_'):
                    net.fit(Xs, Qs, Ys)
                else:
                    net.fit(Xs, Ys)
                # measure the RMSE on the test data
                mse = 0.
                for i in range(N_test):
                    Ypred = net.predict(Xs_test_one_hot[i])
                    Yexp, _ = fsm.label_sequence(Xs_test[i], delta, rho)
                    mse   += np.mean((Ypred.squeeze() - Yexp) ** 2)
                rmse = np.sqrt(mse / N_test)
                params_r['errors'].append(rmse)
                if rmse < min_rmse:
                    min_rmse = rmse
            print('best rmse: %g' % min_rmse)
    # write the results to a JSON file
    with open(hyperparam_path, 'w') as hyperparam_file:
        json.dump(hyperparams, hyperparam_file)

# select best hyperparameters for each model
hyperparams_opt = {}
for model in models:
    min_err = np.inf
    for params_r in hyperparams[model]:
        if np.mean(params_r['errors']) < min_err:
            min_err = np.mean(params_r['errors'])
            hyperparams_opt[model] = params_r
    print('\nSelected the following hyper-parameters for %s' % model)
    for key in hyperparams_opt[model]:
        print('%s: %s' % (key, str(hyperparams_opt[model][key])))

--- repeat 1 of 10 ---
-- model: ESN --
best rmse: 0.265265
-- model: CRJ --
best rmse: 0.393867
-- model: LMU --
best rmse: 0.711471
-- model: RMM_ESN --
best rmse: 0
-- model: RMM_CRJ --
best rmse: 0
-- model: RMM_LMU --
best rmse: 0
--- repeat 2 of 10 ---
-- model: ESN --
best rmse: 0.478202
-- model: CRJ --
best rmse: 0.39117
-- model: LMU --
best rmse: 0.449159
-- model: RMM_ESN --
best rmse: 0
-- model: RMM_CRJ --
best rmse: 0
-- model: RMM_LMU --
best rmse: 0
--- repeat 3 of 10 ---
-- model: ESN --
best rmse: 0.411715
-- model: CRJ --
best rmse: 0.478991
-- model: LMU --
best rmse: 0.820361
-- model: RMM_ESN --
best rmse: 0
-- model: RMM_CRJ --
best rmse: 0
-- model: RMM_LMU --
best rmse: 0
--- repeat 4 of 10 ---
-- model: ESN --
best rmse: 0.370682
-- model: CRJ --
best rmse: 0.558135
-- model: LMU --
best rmse: 0.406913
-- model: RMM_ESN --
best rmse: 0
-- model: RMM_CRJ --
best rmse: 0
-- model: RMM_LMU --
best rmse: 0
--- repeat 5 of 10 ---
-- model: ESN --
best rmse: 0.2754

## Actual Experiment

In [4]:
# set up the number of repeats for the actual experiment
R = 20

# perform the actual experiment in a crossvalidation
errors   = np.zeros((len(models), R))
errors_with_inference = np.zeros((len(models)+1, R))
runtimes = np.zeros((len(models), R))
runtimes_with_inference = np.zeros((len(models)+1, R))

import time

for r in range(R):
    print('--- repeat %d of %d ---' % (r+1, R))
    # sample a finite state machine but exclude trivial ones
    while True:
        delta, rho = fsm.sample_fsm(num_states, num_in_symbols, num_out_symbols)
        if len(np.unique(rho)) > 1:
            break
    for with_inference in [False, True]:
        # generate the according training data
        if with_inference:
            start_time = time.time()
            Xs, Qs, Ys, delta2, rho2 = generate_seq(delta, rho, with_inference)
            # check error that is purely due to automaton learning difficulaties
            mse = 0.
            for i in range(N_test):
                Ypred, _ = fsm.label_sequence(Xs_test[i], delta2, rho2)
                Yexp, _  = fsm.label_sequence(Xs_test[i], delta, rho)
                mse += np.mean((Ypred - Yexp) ** 2)
            rmse = np.sqrt(mse / N_test)
            runtimes_with_inference[-1, r] = time.time() - start_time
            errors_with_inference[-1, r] = rmse
        else:
            Xs, Qs, Ys = generate_seq(delta, rho, with_inference)
        # now iterate over all models
        for model_idx in range(len(models)):
            model = models[model_idx]
            # print('-- model: %s --' % model)
            # set up the model with the best selected hyperparameters
            start_time = time.time()
            net = setup_model(model, hyperparams_opt[model])
            # fit the model to the data
            if model.startswith('RMM_'):
                net.fit(Xs, Qs, Ys)
            else:
                net.fit(Xs, Ys)
            # measure the RMSE on the test data
            mse = 0.
            for i in range(N_test):
                Ypred = net.predict(Xs_test_one_hot[i])
                Yexp, _ = fsm.label_sequence(Xs_test[i], delta, rho)
                mse   += np.mean((Ypred.squeeze() - Yexp) ** 2)
            rmse = np.sqrt(mse / N_test)
            if with_inference:
                runtimes_with_inference[model_idx, r] = time.time() - start_time
                errors_with_inference[model_idx, r] = rmse
            else:
                runtimes[model_idx, r] = time.time() - start_time
                errors[model_idx, r] = rmse

--- repeat 1 of 20 ---
--- repeat 2 of 20 ---
--- repeat 3 of 20 ---
--- repeat 4 of 20 ---
--- repeat 5 of 20 ---
--- repeat 6 of 20 ---
--- repeat 7 of 20 ---
--- repeat 8 of 20 ---
--- repeat 9 of 20 ---
--- repeat 10 of 20 ---
--- repeat 11 of 20 ---
--- repeat 12 of 20 ---
--- repeat 13 of 20 ---
--- repeat 14 of 20 ---
--- repeat 15 of 20 ---
--- repeat 16 of 20 ---
--- repeat 17 of 20 ---
--- repeat 18 of 20 ---
--- repeat 19 of 20 ---
--- repeat 20 of 20 ---


In [5]:
print('--- without inference ---')
for model_idx in range(len(models)):
    print('%s: %g +- %g (took %g seconds)' % (models[model_idx], np.mean(errors[model_idx, :]), np.std(errors[model_idx, :]), np.mean(runtimes[model_idx, :])))
print('--- with inference ---')
for model_idx in range(len(models)):
    print('%s: %g +- %g (took %g seconds)' % (models[model_idx], np.mean(errors_with_inference[model_idx, :]), np.std(errors_with_inference[model_idx, :]), np.mean(runtimes_with_inference[model_idx, :])))
print('inference: %g +- %g (took %g seconds)' % (np.mean(errors_with_inference[-1, :]), np.std(errors_with_inference[-1, :]), np.mean(runtimes_with_inference[-1, :])))

--- without inference ---
ESN: 0.47029 +- 0.160935 (took 0.213089 seconds)
CRJ: 0.604922 +- 0.22382 (took 0.127775 seconds)
LMU: 0.541558 +- 0.187286 (took 0.256654 seconds)
RMM_ESN: 0 +- 0 (took 0.813254 seconds)
RMM_CRJ: 0 +- 0 (took 0.733528 seconds)
RMM_LMU: 0 +- 0 (took 0.820162 seconds)
--- with inference ---
ESN: 0.456468 +- 0.178707 (took 0.199003 seconds)
CRJ: 0.611023 +- 0.217696 (took 0.126003 seconds)
LMU: 0.559275 +- 0.180837 (took 0.252807 seconds)
RMM_ESN: 0.023416 +- 0.0715245 (took 0.812356 seconds)
RMM_CRJ: 0.023416 +- 0.0715245 (took 0.714939 seconds)
RMM_LMU: 0.023416 +- 0.0715245 (took 0.830829 seconds)
inference: 0.023416 +- 0.0715245 (took 0.0138352 seconds)


In [6]:
# store results
np.savetxt('%s_errors.csv' % task, errors.T, delimiter='\t', header='\t'.join(models), comments='')
np.savetxt('%s_runtimes.csv' % task, runtimes.T, delimiter='\t', header='\t'.join(models), comments='')
np.savetxt('%s_errors_with_inference.csv' % task, errors_with_inference.T, delimiter='\t', header='\t'.join(models) + '\tinference', comments='')
np.savetxt('%s_runtimes_with_inference.csv' % task, runtimes_with_inference.T, delimiter='\t', header='\t'.join(models) + '\tinference', comments='')